<a href="https://colab.research.google.com/github/rmadushani/Three_vasopressor_problem/blob/main/eICU/codes/eICU_Three_Vasopressors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###
# @author: R. W. M. A. Madushani
# Created on Dec 20, 2019
###

In [2]:
# Imports for accessing eICU data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# Import libraries
import os
import pandas as pd
import numpy as np

In [3]:
auth.authenticate_user()

In [4]:
project_id = 'sccm-datathon'
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

In [5]:
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# Extract and clean demographic data

In [6]:
# Extract relevant patient data
patient_dat_query = \
"""
SELECT patientHealthSystemStayID, patientUnitStayID, uniquepid, gender, age, ethnicity, unitType, hospitalDischargeLocation, hospitalDischargeStatus
FROM `physionet-data.eicu_crd.patient`
"""

patient_dat = run_query(patient_dat_query)

## We consider only 'MICU', and 'Med-Surg ICU'
patient_dat_new = patient_dat.loc[(patient_dat.unitType=='MICU') | (patient_dat.unitType=='Med-Surg ICU'),: ].copy()

## Convert age>89 strings to numeric to extract patients with age>=18
patient_dat_new['age_int'] = patient_dat_new['age'].apply(lambda x: 90 if x=='> 89' else (-999 if x=='' else int(x)))
## Filter patients with age >= 18
patient_dat_new = patient_dat_new.loc[patient_dat_new.age_int >= 18, :]

## Remove records with missing hospital discharge status i.e., mortality
patient_dat_new = patient_dat_new.loc[patient_dat_new.hospitalDischargeStatus!='', :]
patient_dat_new.head()

,patientHealthSystemStayID,patientUnitStayID,uniquepid,gender,age,ethnicity,unitType,hospitalDischargeLocation,hospitalDischargeStatus,age_int
0,844848,1137569,011-43764,Female,75,Other/Unknown,MICU,Death,Expired,75
1,2462651,3036927,030-57208,Male,51,Caucasian,Med-Surg ICU,Home,Alive,51
2,2481889,3058863,030-13634,Female,31,Caucasian,Med-Surg ICU,Home,Alive,31
3,2494105,3072720,030-53536,Male,58,Caucasian,Med-Surg ICU,Home,Alive,58
4,2496542,3075429,030-59276,Female,68,Caucasian,Med-Surg ICU,Death,Expired,68


In [7]:
## List of ICU stays ids to extract other variables
patientUnitStayIDs = list(patient_dat_new.patientUnitStayID)

# Extract and clean vasopressor medication data

In [8]:
# Extract vasopressor related medication data
vasp_med_query = \
"""
SELECT medicationid, patientunitstayid, drugstartoffset, drugname, drughiclseqno, routeadmin, frequency, drugstopoffset, gtc
FROM `physionet-data.eicu_crd.medication`
WHERE (drugname is null OR (UPPER(drugname) like '%EPINEPHRIN%') OR (UPPER(drugname) LIKE '%ADRENALIN%') OR (UPPER(drugname) LIKE '%PHENYLEPHRIN%') OR (UPPER(drugname) LIKE '%VASOPRESSIN%') OR (UPPER(drugname) like '%DOPAMINE%') OR (UPPER(drugname) like '%ANGIOTENSIN%')) 
"""

med_vasp_dat = run_query(vasp_med_query)

In [9]:
## Extract medication data correponding to filtered patient data
med_vasp_dat = med_vasp_dat.loc[med_vasp_dat.patientunitstayid.isin(patientUnitStayIDs),:]

## Vasopressors considered in the analysisare: 'dopamine', 'vasopressin', 'norepinephrine', 'phenylephrine', 'epinephrine', 'angiotensin'
## HICL codes for vasopressors (was taken from: https://github.com/MIT-LCP/eicu-code/blob/master/concepts/pivoted/pivoted-med.sql)
## Note: 'angiotensin' does not appear in eicu medication data and we were unable to find the corrosponding HICL code for 'angiotensin'
hicl_vasp_dict = {'norepinephrine':[37410, 36346, 2051], 'epinephrine':[37407, 39089, 36437, 34361, 2050], 'dopamine': [2060, 2059], 'phenylephrine': [37028, 35517, 35587, 2087], 'vasopressin':[38884, 38883, 2839]}


## User defined function to impute the vasopressor type (drugname column) using HICL codes
## 'return_key' function maps HICL codes to corresponding vasopressor based on the dictionary "hicl_vasp_dict"
def return_key(val):
  for key, value in hicl_vasp_dict.items():
    if val in value:
      return key 

## User defined function to cleam and impute the vasopressor type (drugname column) using HICL codes
## 'clean_drugname' homogenize the vasopressor names in the drugname column
def clean_drugname(name):
  if name is not None:
    if 'norepinephrine' in name.lower():
      return 'norepinephrine'
    elif 'phenylephrine' in name.lower():
      return 'phenylephrine'
    elif 'epinephrine' in name.lower():
      return 'epinephrine'
    elif 'vasopressin' in name.lower():
      return 'vasopressin'
    elif 'dopamine' in name.lower():
      return 'dopamine'
    elif 'angiotensin' in name.lower():
      return 'angiotensin'


## Homogenize the vasopressor names in the drugname column
med_vasp_dat['drugname_cleaned'] = med_vasp_dat['drugname'].apply(clean_drugname)

## impute the vasopressor type (drugname column) using HICL codes
med_vasp_dat.loc[med_vasp_dat.loc[:, 'drugname'].isnull(), 'drugname_cleaned'] = med_vasp_dat.loc[med_vasp_dat.loc[:, 'drugname'].isnull(), 'drughiclseqno'].apply(return_key)

## impute the vasopressor type (drugname column) using HICL codes
med_vasp_dat.loc[med_vasp_dat.loc[:, 'drugname'].isnull(), 'drugname_cleaned'] = med_vasp_dat.loc[med_vasp_dat.loc[:, 'drugname'].isnull(), 'drughiclseqno'].apply(return_key)

## Drop records with missing vasopressor type
med_vasp_dat_new = med_vasp_dat.loc[~med_vasp_dat.drugname_cleaned.isnull(),:].copy()

## Zero drug start/stop times may not be correct as they may be representing nulls 
## Thereore, we remove those
med_vasp_dat_new = med_vasp_dat_new.loc[~((med_vasp_dat_new.drugstartoffset==0) | (med_vasp_dat_new.drugstopoffset==0)),:]

## Drop records with imposible start-stop times
med_vasp_dat_new = med_vasp_dat_new.loc[med_vasp_dat_new.drugstopoffset > med_vasp_dat_new.drugstartoffset, :]

## Drop records of vasopressor administration before ICU
## Note that if drugstartoffset is negative when drugstopoffset is positive, we do not drop those
## We consider the vasopressor administration time from ICU admission to drug stop time to determine 2hour time crieteria for such records
med_vasp_dat_new = med_vasp_dat_new.loc[med_vasp_dat_new.drugstopoffset	> 0,:]

In [10]:
med_vasp_dat_new.head()

,medicationid,patientunitstayid,drugstartoffset,drugname,drughiclseqno,routeadmin,frequency,drugstopoffset,gtc,drugname_cleaned
76324,94900852,2993662,933,None,34361.0,Miscellaneou,ONE TIME X1,948,0,epinephrine
78881,57739756,1712479,-411,NOREPINEPHRINE,NaN,IV CONT,X1 M1019,111,0,norepinephrine
78950,60470542,1607519,2755,NOREPINEPHRINE,NaN,IV CONT,INFUSE,5487,0,norepinephrine
79707,47536687,1355842,78,phenylephrine,NaN,Each Nostril,Every 6 hours PRN,5777,0,phenylephrine
81416,59184082,1681549,1559,PHENYLEPHRINE,NaN,IV CONT,INFUSE,8261,0,phenylephrine


In [11]:
med_vasp_dat_new.routeadmin.unique() ## Note that there are data of which vasopressor routes of administration are not relevant to the study as we consider infusion drugs

array(['Miscellaneou', 'IV CONT', 'Each Nostril', 'IV', 'IR',
       'CENTRAL IV', 'IVPB', 'SUBQ', 'IM', '.ROUTE', 'See Instruct',
       'Intravenous', 'Intra-articular', 'X', 'PERIPH IV', 'Endotracheal',
       'ENDOSCOPY TU', 'INH', 'INHAL', 'NEBULIZER', 'IntraVENOUS',
       'INTRAVEN.',
       'IV (intravenous)                                                                                    ',
       'INTRAVEN', 'Subcutaneous', 'SUBCUTAN', 'SC', 'Intramuscular',
       'subCUT', 'Inj', 'SQ',
       'IV (injection)                                                                                      ',
       'INTRAVENOUS CONTINUOUS', 'SubQ',
       'IM (injection)                                                                                      ',
       'LVF', 'IJ', 'IV Push', 'INJ', 'SUBCUT', 'IV PUSH', 'IVP',
       'IVDRIP', 'Central Line',
       'IV - continuous infusion (intravenous)                                                              ',
       'INTRAVENOUS', '

In [12]:
## Filter data by route of administration
## Note that we selected these routes according to expert opinion after a thorough investigation of data
roa_list = ['INTRAVEN.', 'INF', 'IV', 'IV CONT', 'Intravenous', 'IntraVENOUS', 'INTRAVENOU', 'IVDRIP',  'INTRAVEN',  'INTRAVENOUS CONTINUOUS', 'IV (intravenous)                                                                                    ', 'IV - continuous infusion (intravenous)                                                              ', 'INTRAVENOUS',  'IVPB',  'CENTRAL IV', 'PERIPH IV', 'Central Line',  'IV - continuous infusion (injection)                                                                ', 'INJ', 'Inj']

med_vasp_dat_new = med_vasp_dat_new.loc[med_vasp_dat_new.routeadmin.isin(roa_list), ['patientunitstayid', 'drugstartoffset', 'drugstopoffset', 'drugname_cleaned']]
med_vasp_dat_new.head()

,patientunitstayid,drugstartoffset,drugstopoffset,drugname_cleaned
78881,1712479,-411,111,norepinephrine
78950,1607519,2755,5487,norepinephrine
81416,1681549,1559,8261,phenylephrine
83880,1625130,81,241,norepinephrine
85161,1607519,6835,9895,norepinephrine


In what follows we extract start and stop times of third vasopressor administration. For that we:

(1) Merge overlapping times of same vasopressor administration per ICU stay

(2) Extract the start and stop times for third vasopressor administered continuously at least for 2 hours

In [13]:
## Define a function to merge overlapping/continuous vasopressor records
## Input: "dat_by_icu_stays" is data from "med_vasp_dat_new" grouped by 'patientunitstayid'
## Output: "df" is a dataframe with merged overlapping times per ICU stay
def merge_vasopressors(dat_by_icu_stays):
  df = pd.DataFrame(columns=['patientunitstayid', 'drugstartoffset', 'drugstopoffset', 'drugname_cleaned'])
  vasp_types = list(dat_by_icu_stays.drugname_cleaned.unique())
  for vasp in vasp_types:
    vasp_dat_temp = dat_by_icu_stays.loc[dat_by_icu_stays.drugname_cleaned == vasp, :].copy()
    vasp_dat_temp.sort_values(by=['drugstartoffset', 'drugstopoffset'], ascending=True, inplace=True)
    vasp_dat_temp.reset_index(drop=True, inplace=True)
    df = df.append(vasp_dat_temp.loc[0,:], ignore_index=True)
    for ind in range(1, (vasp_dat_temp.shape[0])):
      start_time_first = df.loc[df.index[-1], 'drugstartoffset']
      stop_time_first = df.loc[df.index[-1], 'drugstopoffset']
      start_time_second = vasp_dat_temp.loc[ind, 'drugstartoffset']
      stop_time_second = vasp_dat_temp.loc[ind, 'drugstopoffset']
      if (start_time_second <= stop_time_first):
        df.loc[df.index[-1], 'drugstopoffset'] = max(stop_time_first, stop_time_second)    
      else:
        df = df.append(vasp_dat_temp.loc[ind,:], ignore_index=True)
  return df

In [14]:
## Merge overlapping vasopressor administrations
merged_dat = med_vasp_dat_new.groupby(by='patientunitstayid').apply(merge_vasopressors)

merged_dat.reset_index(drop=True, inplace=True)
merged_dat.head()

,patientunitstayid,drugstartoffset,drugstopoffset,drugname_cleaned
0,141168,2121,2390,dopamine
1,141168,2046,2390,norepinephrine
2,141288,-8,3100,norepinephrine
3,141432,5,1210,norepinephrine
4,141585,7406,17745,norepinephrine


In [15]:
## Next we use "merged_dat" to extract third vasopressor times
## Note that we would like to extract third vasopressor times of which 
## vasopressors were administered continuously for at least two hours durong the
## ICU stay. Two hour time period starts from the ICU admission.
## Therefore, we make start time of all vasopressors administration began prior to ICU admission zero.
## However, we still want to keep the original start times for later use.
merged_dat.loc[:, 'drugstartoffset_original'] = merged_dat.loc[:, 'drugstartoffset']
## Change drug start time to ICU admission time when the drug start time is prior to ICU admission
merged_dat.loc[merged_dat.drugstartoffset < 0, 'drugstartoffset'] = 0

## Remove data of which vasopressor administration was less than two hours 
merged_dat = merged_dat.loc[(merged_dat.drugstopoffset-merged_dat.drugstartoffset) >= (2*60), :].copy()
merged_dat.head()

,patientunitstayid,drugstartoffset,drugstopoffset,drugname_cleaned,drugstartoffset_original
0,141168,2121,2390,dopamine,2121
1,141168,2046,2390,norepinephrine,2046
2,141288,0,3100,norepinephrine,-8
3,141432,5,1210,norepinephrine,5
4,141585,7406,17745,norepinephrine,7406


In [16]:
## Define a function to extract third vasopressor time
## Input: "dat" is data from "merged_dat" grouped by 'patientunitstayid'
## Output: "third_vasp_start_time_dat" is a dataframe with third vasopressor times per ICU stay
def third_vasp(dat):
  third_vasp_start_time_dat = pd.DataFrame(columns=['patientunitstayid', 'drugstartoffset', 'drugstopoffset', 'drugname_cleaned', 'drugstartoffset_original', 'first_vasp_drugstartoffset', 'second_vasp_drugstartoffset'])
  for row_id in range(dat.shape[0]-2):
    dat_temp = dat.iloc[row_id:(row_id+3)].copy()
    if ((dat_temp.drugname_cleaned.nunique() == 3) and ((min(dat_temp.drugstopoffset)-max(dat_temp.drugstartoffset)) >= 120)):
      dat_temp[['first_vasp_drugstartoffset']] = dat_temp.iloc[0, 4]
      dat_temp[['second_vasp_drugstartoffset']] = dat_temp.iloc[1, 4]
      third_vasp_start_time_dat = third_vasp_start_time_dat.append(dat_temp.iloc[2, :], ignore_index=True)
      break;
  return third_vasp_start_time_dat

In [17]:
## Extract third vasopressor times
merged_dat.sort_values(by=['patientunitstayid', 'drugstartoffset', 'drugstopoffset'], ascending=True, inplace=True)
third_vasp_time_dat = merged_dat.groupby(by='patientunitstayid').apply(third_vasp)
third_vasp_time_dat.head()

,,patientunitstayid,drugstartoffset,drugstopoffset,drugname_cleaned,drugstartoffset_original,first_vasp_drugstartoffset,second_vasp_drugstartoffset
patientunitstayid,,,,,,,,
144173,0,144173,107,390,vasopressin,107,47,92
145394,0,145394,2982,10012,phenylephrine,2982,222,2982
146133,0,146133,1663,2039,dopamine,1663,43,268
146349,0,146349,677,3286,phenylephrine,677,497,497
147985,0,147985,369,3009,norepinephrine,369,174,279


In [20]:
## Check for any third vasopressor starting time prior to ICU admission
print(sum(third_vasp_time_dat.drugstartoffset_original<0))

126

In [21]:
# Remove third vasopressor start times prior to ICU admission
third_vasp_time_dat = third_vasp_time_dat.loc[third_vasp_time_dat.drugstartoffset_original>=0,:]

In [26]:
## Rename columns of third vasopressor times
third_vasp_time_dat.rename(columns={'drugstartoffset': 'third_vasp_drugstartoffset', 'drugstopoffset': 'third_vasp_drugstartoffset', 'drugstartoffset_original': 'third_vasp_drugstartoffset_original', 'drugname_cleaned': 'drugname'}, inplace=True)

third_vasp_time_dat.reset_index(drop=True, inplace=True)
third_vasp_time_dat.head()

,patientunitstayid,third_vasp_drugstartoffset,third_vasp_drugstartoffset,drugname,third_vasp_drugstartoffset_original,first_vasp_drugstartoffset,second_vasp_drugstartoffset
0,144173,107,390,vasopressin,107,47,92
1,145394,2982,10012,phenylephrine,2982,222,2982
2,146133,1663,2039,dopamine,1663,43,268
3,146349,677,3286,phenylephrine,677,497,497
4,147985,369,3009,norepinephrine,369,174,279
